<img src='data/images/section-notebook-header.png' />

# Vector Space Model (VSM)

The **Vector Space Model (VSM)** is a fundamental concept in Natural Language Processing that represents textual documents as numerical vectors in a high-dimensional space. It is a mathematical framework that allows us to analyze and compare documents based on their semantic similarity. In the VSM, each document is typically represented as a vector where each dimension corresponds to a particular term or word in the document. The value of each dimension in the vector represents the importance or presence of that term in the document. The choice of representation can vary, but common approaches include using term frequency-inverse document frequency (TF-IDF) weights or binary values (indicating term presence or absence).

To build the VSM, a corpus of documents is first constructed, and a vocabulary is formed by extracting unique terms from the documents. Then, each document is transformed into a vector by assigning appropriate weights to the terms based on their occurrences or importance in the document. Once the documents are represented as vectors, various mathematical operations can be performed to analyze the relationships between documents. These operations include measuring the cosine similarity or Euclidean distance between document vectors to determine their similarity or dissimilarity. Vector space models enable tasks such as document classification, information retrieval, document clustering, and more.

One of the advantages of the vector space model is that it allows us to capture the semantic meaning of documents and compare them in a quantitative manner. However, it does not explicitly model the grammar or syntactic structure of the text. Nonetheless, it forms the basis for many advanced NLP techniques and models such as text classification.

## Setting up the Notebook

### Import all Required Packages

In [ ]:
import numpy as np
import pandas as pd
import math

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Define an Example Corpus

Let's define a simple corpus of 5 documents, each containing just one sentence. For convenience, we use the same toy corpus as in the lectures. All sentences have been preprocessed -- that is, all words a lower case, punctuation has been removed, and stop words have been removed.


In [ ]:
documents = ['dog chase cat dog',
             'cat chase cat',
             'car chase tv',
             'dog watch dog tv',
             'dog cat sit car']

# Remember the number of documents for later
N = len(documents)

---

## Calculate Word Counts

In practice, we typically use existing toolkits or packages to convert documents into vectors using the VSM, and we will do so in this notebook as well. However, we also perform the underlying calculations "manually" as well to get a better understanding how this conversion is done. To this end, we need to derive series of statistics from out example document we need through the tutorial:

- `word_list`: a list of all words, i.e., the vocabulary of the corpus

- `word_to_idx`: a dictionary that maps a word to an index; most algorithms don't consider words but indexes/numbers for efficiency reasons. Don't forget, algorithms don't care about words specifically. Note below that the index matches the position in the `word_list`. As such, we can not only map from a word to the index but also from an index to the respective word.

- `doc_counts`: a dictionary that keeps track of how many documents contain a certain word. This later simplifies the calculation of the inverse document frequency (idf)

In [ ]:
vocabulary = set([])
doc_counts = {}

for doc in documents:
    # For each word in the document...
    for term in doc.split():
        # If we haven't seen this word yet...
        if term not in vocabulary:
            # Add term to list
            vocabulary.add(term)
    # for each UNIQUE word in the document...
    for term in set(doc.split()):
        if term not in doc_counts:
            doc_counts[term] = 1
        else:
            doc_counts[term] += 1

# Let's conver to vocabulary from a set to a sorted list
# (not really needed but it ensures that the result matrices match the examples from the lectures)
vocabulary_manual = sorted(vocabulary)            
            
# Calculate and remember size of vocabulary
V = len(vocabulary_manual)             

# Create a dictionary to map a term to a unique index from 0..(V-1)
# We need this to make it easy to access the row of the term-document matrix
term_to_idx = { k:v for v, k in enumerate(sorted(vocabulary)) }
          
            
# Let's see how the 3 statistics look like            
print(vocabulary_manual)
print(term_to_idx)
print(doc_counts)
print()
print("Size of vocabulary: {}".format(V))

---

## Term-Document Matrix 

The term-document matrix is a key component of the vector space model in NLP. It is a two-dimensional matrix that represents the occurrences or weights of terms in a collection of documents. The rows of the matrix correspond to the terms or words in the vocabulary, and the columns represent the documents in the corpus.

In a term-document matrix, each cell represents the frequency, presence, or weight of a particular term in a specific document. The choice of representation depends on the specific weighting scheme used, such as term frequency (TF), inverse document frequency (IDF), or TF-IDF.

The term-document matrix serves as the foundation for various NLP tasks. It allows us to analyze the relationships between terms and documents, calculate document similarity, perform text classification, and extract important features from the corpus. By transforming text into a numerical representation, it enables the application of various mathematical and statistical techniques to process and analyze textual data.

### Term Frequency Term-Document Matrix (manually)

The term frequency term-document matrix, often abbreviated as TF term-document matrix, is a specific type of term-document matrix where the values in each cell represent the frequency of a term in a document. It is a simple and commonly used representation that captures the occurrence information of terms within the corpus.

In a TF term-document matrix, the frequency of a term is typically calculated using the raw count of occurrences within a document. Each cell in the matrix represents the number of times a term appears in a specific document. We know the size of the term-document matrix: The size of the vocabulary defines the number or rows, and the number of documents defines the number of columns. Both this information we already have, so we can initialize the term-document matrix using a numpy array with the corresponding dimensions.

In [ ]:
# V = size of vocabulary; N = number of documents
tf_tdm_manual =  np.zeros(shape=(V, N), dtype=np.int8)

print(tf_tdm_manual)

Of course, all values are 0 so far. We now need to fill this matrix with the respective term frequencies. The code cell below accomplishes this. 

In [ ]:
for doc_idx, doc in enumerate(documents):
    for term in doc.split():
        # We use the index mapping to find the correct index of a word.
        tf_tdm_manual[term_to_idx[term], doc_idx] += 1
            
print(tf_tdm_manual)

Just to have a nicer visualization, we can convert our TF term-document matrix into a `pandas` dataframe. With respect to any further processing or use of the matrix, this is completely unnecessary. We only do it to have a more nicely printed matrix.

In [ ]:
pd.DataFrame(tf_tdm_manual, index=list(vocabulary_manual), columns=['d{}'.format(c) for c in range(1, N+1)])

The output should match the TF term-document matrix from the lectures.

### Term Frequency Document Word Matrix (using scikit-learn)

Since transforming a corpus into a term-document matrix, all popular NLP toolkits provide respective class and/or methods. The `CountVectorizer` is a class in the scikit-learn library. It is a feature extraction tool specifically designed for converting text documents into numerical feature vectors based on the VSM. The CountVectorizer implements the process of tokenizing, counting, and creating a term-document matrix based on the term frequencies.

Here's a brief overview of how `CountVectorizer` works:

* **Text Preprocessing:** Before using `CountVectorizer`, it's common to perform preprocessing steps such as lowercasing, removing punctuation, handling stopwords, and other text normalization techniques. These steps help to clean and standardize the text data. This has already been done for our example corpus.

* **Creating an Instance:** To use `CountVectorizer`, you first create an instance of the `CountVectorizer` class, optionally specifying various parameters that control the vectorization process. Some common parameters include `lowercase`, `stop_words`, `tokenizer`, and `ngram_range`, among others.

* **Fitting and Transforming:** Once the `CountVectorizer` instance is created, you can fit it on your corpus of documents using the `fit_transform()` method. This step learns the vocabulary from the corpus and transforms the input documents into a term-document matrix.

* **Obtaining the Term-Document Matrix:** After fitting the `CountVectorizer`, you can access the term-document matrix using the `transform()` method. This method takes new text documents as input and returns their corresponding numerical representation based on the learned vocabulary.

Let's define an instance of the `CountVectorizer` using its default values (cf. the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for more details)


In [ ]:
count_vectorizer = CountVectorizer()

The method `fit_transform()` takes the corpus as input and performs the generation of the TF term-document matrix.

In [ ]:
tf_tdm_sklearn = count_vectorizer.fit_transform(documents)
vocabulary_sklearn = count_vectorizer.get_feature_names_out()

print(vocabulary_sklearn)

Again, we can use `pandas` to conveniently display the resulting TF term-document matrix.

In [ ]:
pd.DataFrame(tf_tdm_sklearn.A.T, index=list(vocabulary_sklearn), columns=['d{}'.format(c) for c in range(1, N+1)])

Of course, the 2 matrix outputs should be exactly the same.

### TF-IDF Term-Document Matrix (manually)

The TF term-document matrix provides a basic representation of the term frequencies in the corpus. However, it doesn't take into account the relative importance of terms in the entire document collection. To address this limitation and give more weight to terms that are relatively rare but informative, other techniques like inverse document frequency (IDF) or TF-IDF weighting are often applied in conjunction with the TF term-document matrix.

The TF-IDF (Term Frequency-Inverse Document Frequency) term-document matrix takes into account the importance of each term in a document relative to the entire collection of documents. It consists of rows representing terms and columns representing documents. Each entry in the matrix represents the weight of a term in a document, computed using the TF-IDF formula.

Here's a step-by-step explanation of how the TF-IDF term-document matrix is constructed:

* **Term Frequency (TF):** Calculate the frequency of each term (word) in each document. This can be done by counting the number of occurrences of each term in a document or by using other weighting schemes like log normalization to mitigate the impact of term frequency variations.

* **Inverse Document Frequency (IDF):** Calculate the inverse document frequency of each term. IDF quantifies the rarity of a term across the entire document collection. Terms that appear in many documents have lower IDF values, while terms that appear in a few documents have higher IDF values. 

* **TF-IDF:** Multiply the term frequency (TF) of each term in a document by its inverse document frequency (IDF) to obtain the TF-IDF weight for that term in that document. This step assigns higher weights to terms that are frequent in a document but rare in the entire collection, emphasizing their importance.

* **Building the matrix:** Each document is represented as a vector where each element corresponds to the TF-IDF weight of a term in that document. All the document vectors are then stacked together to form the TF-IDF term-document matrix.

An element in the TF-IDF term-document matrix (tf = term frequency; idf = inverse document frequency) for a term $t_i$ and a document $d_j$ is calculated as follows:

$$tfidf(t_i, d_j) = tf(t_i, d_j) \cdot idf(t_i, d_j)$$

with

$$ tf(t_i, d_j) = \frac{number\ of\ times\ t_i\ appears\ in\ d_j}{total\ number\ of\ terms\ in\ d_j} $$

and

$$ idf(t_i, d_j) = \log{\frac{total\ number\ of\ documents}{number\ of\ terms\ containing\ t_i}} $$


**IMPORTANT**: In the following, we calculate the term frequency simply as $tf(t_i, d_j) = number\ of\ times\ t_i\ appears\ in\ d_j$. This is how `scikit-learn` does it, and we want to end up with the same result for comparison. 

### TF-IDF Term-Document Matrix (manually)

Like for the TF term-document matrix above, we first calculate the TF-IDF Term-Document Matrix "manually", i.e., without the use of any existing method from a package such as `sklearn`. Compared to simply calculating the term frequencies, we now also have to consider the document frequencies. We already calculated those values at the beginning, and they are now available in the dictionary `doc_counts`.


In [ ]:
# V = size of vocabulary; N = number of documents
tf_idf_manual =  np.zeros(shape=(V, N))

for doc_idx, doc in enumerate(documents):
    # Split the document into list of words
    terms = doc.split()
    # Generate a set of words from the list (i.e., no duplicates)
    terms_set = set(terms)
    # For each unique word in the document (i.e., the word set)
    for term in vocabulary_manual:
        # Calculate tf; count() is an in-built method the returns the number of occurrences of an item in a list
        tf = terms.count(term) 
        # Calculate the document frequence df which we already did when building the statistics
        df = doc_counts[term]
        # Calculate the idf; the +1 are for smoothing to match the results of the scikit-learn methods
        idf = math.log( (N + 1) / (df + 1) ) + 1
        # Finally, caluclate the tf-idf value
        tfidf = tf * idf
        # Set corresponding value in TF-IDF term-document matrix
        tf_idf_manual[term_to_idx[term], doc_idx] = tfidf

print(tf_idf_manual)

Again, we can convert the `numpy` array into a `pandas` dataframe for a nicer visualization of the TF-IDF term-document matrix.

In [ ]:
pd.DataFrame(tf_idf_manual, index=list(vocabulary_manual), columns=['d{}'.format(c) for c in range(1, N+1)])

Please keep in mind that this result does **not** match the TF-IDF term-document matrix shown in the lectures since we used a different formula to calculate the TF-IDF values; we use the formula implemented by `sklearn` by default (see below) to make it easier to compare the results.

### TF-IDF Term-Document Matrix (using scikit-learn)

The `TfidfVectorizer` is a class in the scikit-learn library specifically designed for text feature extraction and vectorization using the TF-IDF term weighting scheme. The `TfidfVectorizer` class in `scikit-learn` combines the functionality of both `CountVectorizer` and `TfidfTransformer`. It performs two main steps:

* **Text preprocessing and tokenization:** It takes a collection of raw text documents as input and preprocesses them by performing tokenization, lowercasing, and removing stop words, punctuation, and other unwanted characters. This step is similar to what CountVectorizer does.

* **TF-IDF transformation:** It computes the TF-IDF term weights for each term in the preprocessed documents. The TF-IDF formula used in TfidfVectorizer is slightly different from the traditional formula mentioned earlier, as scikit-learn applies additional normalization and smoothing techniques to improve the performance.

Let's define an instance of the `TfidfVectorizer` using its default values (cf. the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) for more details).

In [ ]:
tfidf_vectorizer = TfidfVectorizer(norm=None)

The method `fit_transform()` takes the corpus as input and performs the generation of the TF-IDF term-document matrix.

In [ ]:
tfidf_tdm_sklearn = tfidf_vectorizer.fit_transform(documents)
vocabulary_sklearn = tfidf_vectorizer.get_feature_names_out()

For an easy comparison, we print the result matrix again by means of a `pandas` dataframe.

In [ ]:
pd.DataFrame(tfidf_tdm_sklearn.A.T, index=list(vocabulary_sklearn), columns=['d{}'.format(c) for c in range(1, N+1)])

---

## Beyond Unigrams (i.e., single terms)

All `scikit-learn` vectorizers allow users to specify a wide range of parameters. A very important one is `ngram_range` which allows to specify to not only use terms/tokens (1-grams, unigrams) but also n-gram of larger sizes. Note that n-grams larger than 5 are typically not recommended, but it generally depends on the specific applications.

For a quick example, let's generate the TF term-document matrix for our toy corpus considering unigrams and bigrams. The steps are exactly the same as above; we only need to specify the considered ngrams by setting the `ngram_range` parameter value which specifies the lower and upper length for the ngrams to be considered.


In [ ]:
ngram_count_vectorizer = CountVectorizer(ngram_range=(1,2))  # Default is ngram_range=(1,1), i.e., individual words

ngram_tf_model = ngram_count_vectorizer.fit_transform(documents)
ngram_vocabulary = ngram_count_vectorizer.get_feature_names_out()

Let's see how this new TF term-document matrix looks like:

In [ ]:
pd.DataFrame(ngram_tf_model.A.T, index=list(ngram_vocabulary), columns=['d{}'.format(c) for c in range(1, N+1)])

Of course, we now have many more rows since we not only consider unigrams but also bigrams as terms in our matrix.

---

## Summary 

The vector space model (VSM) is a mathematical framework for representing text documents as vectors in a high-dimensional space. It forms the basis for various information retrieval and text mining techniques. In the VSM, each document is represented as a vector, and each dimension of the vector corresponds to a term or word in the document collection. The values in the vector represent the importance or weight of each term in the document.

The vector space model allows for efficient and flexible retrieval of documents based on their relevance to a query. Similarity between documents or between a document and a query can be measured using various distance metrics, such as cosine similarity. By mapping documents into a vector space, the VSM provides a quantitative representation of text, enabling computational techniques to analyze and compare documents based on their content. The vector space model is widely used in applications such as information retrieval, document clustering, text classification, and recommendation systems.